In [138]:
import copy
import random
import time

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

class GameOfLife:

    def __init__(self, grid=None):
        self.grid = grid

    @classmethod
    def random(cls, lines=1, rows=1, seed=1):
        random.seed(seed)
        grid = []
        grid = [None] * lines

        for y in range(0, lines):
            grid[y] = list(random.choice([True, False]) for i in range(rows))
            
        return cls(grid)
    
    @classmethod
    def fromImage(cls, file):
        img = mpimg.imread(file)
        grid = []
        grid = [None] * len(img)
        
        for y in range(0, len(img)):
            grid[y] = list(map(lambda pixel: True if pixel[0] == 0 else False, img[y]))
        
        return cls(grid)

    def tick(self):
        new_grid = copy.deepcopy(self.grid)
        
        # TODO: Magic!

        self.grid = new_grid
        
from IPython.display import display
import ipywidgets as widgets
import time

def loop(game, iterations):
    out = widgets.HTML()
    display(out)
    
    for i in range(0, iterations):
        table = '<table class="game-of-life" style="border: none; padding: 0 0 0 0;">'
        
        aliveCell = '<td height="15px" width="15px" style="background:black;"></td>'
        deadCell  = '<td height="15px" width="15px" style="border:none;"></td>'
        for y in range(0, len(game.grid)):
            table += '<tr>'
            row = game.grid[y]
            row = list(map(lambda cell: aliveCell if cell is True else deadCell, row))
            table += ''.join(row)
            table += '</tr>'
        
        table += '</table>'
        out.value = table
        
        time.sleep(0.250)
        game.tick()

# Conway's Game of Life

# John Conway

* [Surreële getallen](https://www.youtube.com/watch?v=mPn2AdMH7UQ)
* [De Monster groep](https://www.youtube.com/watch?v=jsSeoGpiWsw)
* [De Rij van Conway en Conway's constante](https://nl.wikipedia.org/wiki/Rij_van_Conway)

# Game of Life

In [124]:
game = GameOfLife.random(20, 20, 1)
loop(game, 100)

HTML(value='')

## Regels

1. Een levende cell met 2 of 3 buren overleeft;
2. Een dode cell met 3 buren wordt levend;
3. Alle andere cellen gaan dood.

## Oscillator

In [126]:
game = GameOfLife([[False, False, False], [True, True, True], [False, False, False]])
loop(game, 10)

HTML(value='')

## Stilleven

In [125]:
game = GameOfLife([[False, False, False, False], [False, True, True, False], [False, True, True, False], [False, False, False, False]])
loop(game, 10)

HTML(value='')

## Ruimteschip

In [139]:
game = GameOfLife.fromImage('spaceship.png')
loop(game, 100)

HTML(value='')

Meer patronen vind je op: [https://en.wikipedia.org/wiki/Conway's_Game_of_Life#Examples_of_patterns](https://en.wikipedia.org/wiki/Conway's_Game_of_Life#Examples_of_patterns)

# Python 101

# Hello World

In [ ]:
print("Hello World")

## If-statement

In [ ]:
talkingToWorld = True
if talkingToWorld:
    print("Hello World")
else:
    print("Hello..")

# For-Loops

In [ ]:
for x in range(0, 10):
    print(x)

# Arrays

In [ ]:
array = [1, 1, 2, 3, 5, 8, 13]

print(array[0])
print(array[6])

In [ ]:
len(array)

In [ ]:
grid = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

print(grid[0][0])
print(grid[0][1])
print(grid[1][1])

# Arrays

In [ ]:
array[0] = 42
print(array)

In [ ]:
for x in range(0, len(array)):
    print("Element {} is {}".format(x, array[x]))

In [140]:
game = GameOfLife.fromImage("goslin_glider.png")
loop(game, 100)

FileNotFoundError: [Errno 2] No such file or directory: 'goslin_glider.png'